-- Modelo Conceitual


![](img/ModeloConceitual.png)

-- Modelo Lógico


![](img/ModeloLogico.png)

In [2]:
%defaultDatasource jdbc:h2:mem:db

In [13]:
-- Criação das Tabelas e Importação dos Dados

DROP TABLE IF EXISTS Paciente;
DROP TABLE IF EXISTS RealizaDiagnostico;
DROP TABLE IF EXISTS Diagnostico;

CREATE TABLE Paciente (
  id VARCHAR(6) NOT NULL,
  age INT NOT NULL,
  pregnancies  INT,
  PRIMARY KEY(id)
) AS SELECT
    idPaciente,
    Age,
    Pregnancies
FROM CSVREAD('sheets/Paciente.csv');

CREATE TABLE Diagnostico (
  id VARCHAR(6) NOT NULL,
  glucose INT,
  bloodPressure INT,
  skinThickness INT,
  insulin INT,
  bmi FLOAT,
  pedigreeFunction FLOAT,
  outcome BOOLEAN,
  PRIMARY KEY(id)
) AS SELECT
    idDiagnostico,
    Glucose,
    BloodPressure,
    SkinThickness,
    Insulin,
    BMI,
    DiabetesPedigreeFunction,
    Outcome
FROM CSVREAD('sheets/Diagnostico.csv');

CREATE TABLE RealizaDiagnostico (
  idPaciente VARCHAR(6) NOT NULL,
  idDiagnostico VARCHAR(6) NOT NULL,
  FOREIGN KEY(idPaciente)
    REFERENCES Paciente(id)
      ON DELETE NO ACTION
      ON UPDATE NO ACTION,
  FOREIGN KEY(idDiagnostico)
    REFERENCES Diagnostico(id)
      ON DELETE NO ACTION
      ON UPDATE NO ACTION,
) AS SELECT
    idPacient,
    idDiagnostico
FROM CSVREAD('sheets/RealizaDiagnostico.csv');

In [15]:
SHOW TABLES;

In [26]:
-- Pacientes que fizeram diagnóstico
SELECT DISTINCT P.id, P.age
FROM Paciente P, RealizaDiagnostico RD
WHERE P.id = RD.idPaciente

In [18]:
-- Média de idade dos pacientes que fizeram diagnóstico
SELECT AVG(P.age)
FROM Paciente P, RealizaDiagnostico RD
WHERE P.id = RD.idPaciente

33

In [21]:
-- Média do BMI dos pacientes que fizeram diagnóstico e foram diagnosticados com diabetes
SELECT AVG(D.bmi)
FROM Paciente P, RealizaDiagnostico RD, Diagnostico D
WHERE P.id = RD.idPaciente
    AND D.outcome = 1

35.14253731344829

In [23]:
-- Média da pressão sanguínea das pacientes que passaram por pelo menos 3 partos e foram diagnosticadas com diabetes
SELECT AVG(D.bloodPressure)
FROM Paciente P, RealizaDiagnostico RD, Diagnostico D
WHERE P.id = RD.idPaciente
    AND D.outcome = 1
    AND P.pregnancies >= 3

70

In [25]:
-- Pacientes que tem idade acima de 50 anos, pressão maior que a da média e que não foram diagnosticados com diabetes
SELECT DISTINCT P.id
FROM Paciente P, RealizaDiagnostico RD, Diagnostico D
WHERE P.id = RD.idPaciente
    AND P.age > 50
    AND D.outcome = 0
    AND D.bloodPressure > SELECT AVG(SELECT AVG(D.bloodPressure)
                                    FROM Paciente P, RealizaDiagnostico RD, Diagnostico D
                                    WHERE P.id = RD.idPaciente
                                    )